In [1]:
%pip install networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.5 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [ ]:
# %%

import plotly.graph_objects as go
import networkx as nx
import numpy as np

def plot_network_organic(econ, num_sample_d=30):
    """
    Plota a rede com layout orgânico (force-directed), onde a posição
    depende das conexões, não de camadas fixas.
    """
    
    # --- 1. SELEÇÃO DE DADOS ---
    limit = min(num_sample_d, econ.params.N_d)
    d_indices = np.arange(econ.params.N_d)[:limit]
    
    # Sets para garantir unicidade
    u_indices_set = set()
    z_indices_set = set()
    
    # Listas de arestas (tuplas de strings para o NetworkX)
    edges_u_d = [] 
    edges_z_d = []
    edges_z_u = [] 
    
    # A. Mapear conexões de D
    for i in d_indices:
        # Fornecedores de i
        suppliers = econ.supplier[i]
        for u in suppliers:
            u_indices_set.add(u)
            edges_u_d.append((f'U{u}', f'D{i}')) # IDs únicos como string
            
        # Bancos de i
        banks = econ.bank_links[i]
        for z in banks:
            z_indices_set.add(z)
            edges_z_d.append((f'Z{z}', f'D{i}'))
            
    u_indices = list(u_indices_set)
    
    # B. Mapear conexões de U (se existirem)
    if hasattr(econ, 'bank_links_u') and econ.bank_links_u is not None:
        for u in u_indices:
            banks_of_u = econ.bank_links_u[u]
            for z in banks_of_u:
                z_indices_set.add(z)
                edges_z_u.append((f'Z{z}', f'U{u}'))
    
    z_indices = list(z_indices_set)
    
    # --- 2. CÁLCULO DO LAYOUT (FÍSICA) ---
    G = nx.Graph()
    
    # Adicionar nós
    G.add_nodes_from([f'U{u}' for u in u_indices])
    G.add_nodes_from([f'D{d}' for d in d_indices])
    G.add_nodes_from([f'Z{z}' for z in z_indices])
    
    # Adicionar arestas (Isso define a física da atração)
    G.add_edges_from(edges_u_d)
    G.add_edges_from(edges_z_d)
    G.add_edges_from(edges_z_u)
    
    # Calcular posições (Spring Layout = Mola)
    # k: Distância ideal entre nós (aumente para espalhar mais)
    # iterations: Quantas vezes simular a física
    pos = nx.spring_layout(G, k=0.5, iterations=50, seed=42)
    
    # --- 3. PLOTAGEM (PLOTLY) ---
    fig = go.Figure()

    # Função auxiliar para desenhar arestas
    def add_edge_trace(edge_list, color, dash, name, width=1):
        x_coords = []
        y_coords = []
        for node1, node2 in edge_list:
            if node1 in pos and node2 in pos:
                x0, y0 = pos[node1]
                x1, y1 = pos[node2]
                x_coords.extend([x0, x1, None])
                y_coords.extend([y0, y1, None])
        
        fig.add_trace(go.Scatter(
            x=x_coords, y=y_coords,
            line=dict(width=width, color=color, dash=dash),
            hoverinfo='none',
            mode='lines',
            name=name
        ))

    # Desenhar as conexões
    add_edge_trace(edges_u_d, '#888', 'solid', 'Bens (U-D)')
    add_edge_trace(edges_z_d, 'red', 'dot', 'Crédito (Z-D)')
    add_edge_trace(edges_z_u, 'orange', 'dot', 'Crédito (Z-U)')

    # Função auxiliar para desenhar nós
    def add_node_trace(indices, prefix, color, symbol, name):
        node_x = []
        node_y = []
        text_labels = []
        
        for idx in indices:
            node_id = f'{prefix}{idx}'
            if node_id in pos:
                x, y = pos[node_id]
                node_x.append(x)
                node_y.append(y)
                text_labels.append(node_id)
        
        fig.add_trace(go.Scatter(
            x=node_x, y=node_y,
            mode='markers', # Tirei '+text' para não poluir, mas pode por de volta
            marker=dict(symbol=symbol, size=15, color=color, line=dict(color='black', width=1)),
            text=text_labels,
            hoverinfo='text',
            name=name
        ))

    # Desenhar os nós (mantendo suas formas e cores originais)
    add_node_trace(u_indices, 'U', 'lightgreen', 'square', 'Upstream')
    add_node_trace(d_indices, 'D', 'lightblue', 'circle', 'Downstream')
    add_node_trace(z_indices, 'Z', 'yellow', 'triangle-up', 'Bancos')

    fig.update_layout(
        title="Rede Financeira (Layout Orgânico)",
        showlegend=True,
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='white',
        height=700
    )
    
    fig.show()

# No seu main:
# plot_network_organic(econ, num_sample_d=50)# ============================================================
if __name__ == "__main__":
    # ... seu código de inicialização anterior ...
    # econ = Economy(p, ...)
    # econ.supplier = ...
    # econ.bank_links = ...
    
    # Chama a função de plotagem com uma amostra de 10 firmas Downstream
    plot_network_organic(econ, num_sample_d=30)
# %%
import sys
import subprocess

# Isso vai forçar a instalação no mesmo Python que está rodando o script
subprocess.check_call([sys.executable, "-m", "pip", "install", "networkx"])

import networkx as nx
print("NetworkX instalado e importado com sucesso!")
# %%


In [ ]:
# simulacao bad debt agg
mean_baddebt = np.mean(results_BD, axis=0)
std_baddebt = np.std(results_BD, axis=0)
t_index = np.arange(1, 1001)
fig = go.Figure()
for k in range(min(10, len(results_BD))):
    fig.add_trace(go.Scatter(
        x=t_index, 
        y=results_BD[k, :],
        mode='lines',
        line=dict(width=1, color='rgba(0,0,255,0.2)'), # Azul transparente
        showlegend=False
    ))

# Plotar a MÉDIA (Grossa e sólida)
fig.add_trace(go.Scatter(
    x=t_index,
    y=mean_baddebt,
    mode='lines',
    name='Média (Monte Carlo)',
    line=dict(width=4, color='red')
))

fig.update_layout(
    title=f"Monte Carlo: Bad Debt Agregado (Agregado de 100 simulações)",
    xaxis_title="Tempo (t)",
    yaxis_title="Bad Debt (Valor Monetário)"
)

fig.show()